# What's this?

This notebook tells you examples that SoftDTW works as a distance metric in Kernel functions.

In [6]:
import numpy as np
import sys
sys.path.append("../")
sys.path.append(".")
from model_criticism_mmd import ModelTrainerTorchBackend, MMD, ModelTrainerTheanoBackend, TwoSampleDataSet
from model_criticism_mmd.backends import kernels_torch

In [41]:
np.random.seed(10)

In [42]:
import torch
device_obj = torch.device('cpu')

In [44]:
x_sample, y_sample, x_time_length, y_time_length = 500, 500, 10, 10
# input is (n-sample, n-time-series, n-features)
x_train = torch.tensor(np.random.normal(3, 0.5, size=(x_sample, x_time_length)))
y_train = torch.tensor(np.random.normal(10, 5.5, size=(y_sample, y_time_length)))
x_val = torch.normal(50, 50, (40, x_time_length))
y_val = torch.normal(10, 5.5, (40, y_time_length))

In [45]:
from model_criticism_mmd.backends import kernels_torch
from model_criticism_mmd import MMD, ModelTrainerTorchBackend
from model_criticism_mmd.models import TwoSampleIterDataSet, TwoSampleDataSet

# RBFKernel with SoftDTW does not go advance in trainings

In [46]:
dataset_train = TwoSampleIterDataSet(x=x_train, y=y_train, device_obj=device_obj)
dataset_val = TwoSampleIterDataSet(x=x_val, y=y_val, device_obj=device_obj)

2021-07-12 09:45:54,540 - model_criticism_mmd.logger_unit - INFO - Saving data into /tmp/tmpr04w6unj/data.h5
2021-07-12 09:45:54,629 - model_criticism_mmd.logger_unit - INFO - Saving data into /tmp/tmpiq12tquf/data.h5


In [47]:
rbf_euclide = kernels_torch.BasicRBFKernelFunction(device_obj=device_obj, opt_sigma=False)
trainer_euclide = ModelTrainerTorchBackend(MMD(kernel_function_obj=rbf_euclide, device_obj=device_obj),
                                       device_obj=device_obj)
trained_euclide = trainer_euclide.train(dataset_training=dataset_train,
                                dataset_validation=dataset_val,
                                num_epochs=50,
                                batchsize=30)

2021-07-12 09:45:56,918 - model_criticism_mmd.logger_unit - INFO - Validation(mean over batch) at 0. MMD^2 = 0.1333962231874466, ratio = [1333.96223187] obj = [-7.05196892]
2021-07-12 09:45:57,859 - model_criticism_mmd.logger_unit - INFO -      5: [avg train] MMD^2 0.9681684585907457 obj [-4.91859201] val-MMD^2 0.1395849883556366 val-ratio [69.3503029] val-obj [-4.13142531]  elapsed: 0.0
2021-07-12 09:46:01,354 - model_criticism_mmd.logger_unit - INFO -     25: [avg train] MMD^2 0.9695693903505327 obj [-4.92029522] val-MMD^2 0.1394636034965515 val-ratio [70.48617406] val-obj [-4.14321751]  elapsed: 0.0
2021-07-12 09:46:05,654 - model_criticism_mmd.logger_unit - INFO -     50: [avg train] MMD^2 0.9644243401093439 obj [-4.88183376] val-MMD^2 0.13969430327415466 val-ratio [74.40926752] val-obj [-4.15136473]  elapsed: 0.0


In [ ]:
kernel_function = kernels_torch.SoftDtwKernelFunctionTimeSample(gamma=0.5,
                                                                post_normalize=False,
                                                                opt_sigma=False)
trainer_softdtw = ModelTrainerTorchBackend(MMD(kernel_function_obj=kernel_function, device_obj=device_obj),
                                       device_obj=device_obj)
trained_obj_softdtw = trainer_softdtw.train(dataset_training=dataset_train,
                                            dataset_validation=dataset_val,
                                            num_epochs=50,  # optimization with SoftDTW takes looong time. I set small epoch in the notebook.
                            batchsize=30)

2021-07-12 10:03:16,681 - model_criticism_mmd.logger_unit - INFO - Validation(mean over batch) at 0. MMD^2 = -0.5443306565284729, ratio = [-7.78736531] obj = [13.81551056]
2021-07-12 10:04:04,885 - model_criticism_mmd.logger_unit - INFO -      5: [avg train] MMD^2 -0.0014169987989589572 obj [13.81551056] val-MMD^2 -0.5443306565284729 val-ratio [-7.78736531] val-obj [13.81551056]  elapsed: 0.0


In [ ]:
# https://archive.ics.uci.edu/ml/datasets/Ozone+Level+Detection